In [1]:
# Importar librerías

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
from bs4 import BeautifulSoup
from datetime import datetime
import yfinance as yf
import time
from time import sleep
from textblob import TextBlob

# Librerías de Selenium

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


In [61]:
# Scrolls

def _count_needed_scrolls(browser, infinite_scroll, numOfPost):
    if infinite_scroll:
        lenOfPage = browser.execute_script(
            "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return lenOfPage;"
        )
    else:
        # roughly 8 post per scroll kindaOf
        lenOfPage = int(numOfPost / 8)
    return lenOfPage


def _scroll(browser, infinite_scroll, lenOfPage):
    lastCount = -1
    match = False

    while not match:
        if infinite_scroll:
            lastCount = lenOfPage
        else:
            lastCount += 1

        # wait for the browser to load, this time can be changed slightly ~3 seconds with no difference, but 5 seems
        # to be stable enough
        time.sleep(1)

        if infinite_scroll:
            lenOfPage = browser.execute_script(
                "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return "
                "lenOfPage;")
        else:
            browser.execute_script(
                "window.scrollTo(0, document.body.scrollHeight);var lenOfPage=document.body.scrollHeight;return "
                "lenOfPage;")

        if lastCount == lenOfPage:
            match = True
            

In [63]:
# Reuters.com

def scrape_reuters(tag, numOfPost, infinite_scroll=False):
    news = {
        "titles" : [],
        "dates" : [],
        "par" : [],
        "font" : [],
        "url" : []
    }

    my_url = "https://www.reuters.com/site-search/?query=" + tag
    option = Options()
    option.headless = False
    driver = webdriver.Chrome(options=option)
    driver.get(my_url)
    driver.maximize_window()

    # Entrar en cada noticia
    counter_2 = 0
    counter_3 = 0
    for n in range(numOfPost):
        counter = 0
        while True:
            if counter_2 == 10:
                counter_2 = 0
                counter_3 += 1
                xpath = WebDriverWait(driver, 2).until(
                      EC.element_to_be_clickable((By.XPATH,
                      "/html/body/div[1]/div/div[2]/div[2]/div/div/div[3]/button[2]")))
                xpath.click()
                print("change ", counter_3)
                sleep(3)
            try:
                # Length of page
                lenOfPage = _count_needed_scrolls(driver, infinite_scroll, n - (counter_3 * 10) +1)
                _scroll(driver, infinite_scroll, lenOfPage)
                
                xpath = WebDriverWait(driver, 10).until(
                      EC.visibility_of_element_located((By.XPATH,
                      f"/html/body/div[1]/div/div[2]/div[2]/div/div/div[2]/ul/li[{n - (counter_3 * 10) +1}]/div/div[1]/a")))
                href = xpath.get_attribute('href')
                news['url'].append(href)

                driver.get(href)
                sleep(2)

                # Dentro de la noticia, obtener título, fecha y párrafo de interés

                print("Inside", n, counter_2)
                # HTML
                html = BeautifulSoup(driver.page_source, 'html.parser')

                # Obtener título
                title = html.find(class_="Text__text___3eVx1j Text__dark-grey___AS2I_p Text__medium___1ocDap Text__heading_2___sUlNJP Heading__base___1dDlXY Heading__heading_2___3f_bIW").text
                news["titles"].append(title)

                # Obtener fecha
                date = html.find(class_="DateLine__date___12trWy").text
                news["dates"].append(date)

                # Obtener contenido
                content_list = []
                content = html.findAll("p")
                for p in content:
                    if tag in p.text:
                        content_list.append(p.text)
                    else:
                        continue
                news["par"].append(content_list)
                
                # Obtener fuente
                font = html.find(class_="Text__text___3eVx1j Text__dark-grey___AS2I_p Text__medium___1ocDap Text__heading_4___Ks3Gpt Heading__base___1dDlXY Heading__heading_4___-kdQkt").span.a.text
                news["font"].append(font)
                
                counter_2 += 1
                # Volver
                driver.back()
                break
                
            except Exception as exception:
                if counter < 1:
                    print(exception.__class__.__name__)
                    counter += 1
                    sleep(1)
                    continue
                else:
                    counter_2 += 1
                    break
                    
    
    # Cerrar ventana
    driver.quit()
    
    # Return
    return news

In [62]:
# Obteniendo las noticias

tag = "nasdaq"
n = 200
news = scrape_reuters(tag, n)

print(len(news['par']) / n)

Inside 0 0
Inside 1 1
Inside 2 2
TimeoutException
Inside 4 4
Inside 5 5
Inside 6 6
TimeoutException
Inside 8 8
Inside 9 9
change  1
Inside 10 0
Inside 11 1
Inside 12 2
Inside 13 3
Inside 14 4
Inside 15 5
TimeoutException
Inside 17 7
Inside 18 8
TimeoutException
change  2
Inside 20 0
Inside 21 1
TimeoutException
TimeoutException
Inside 24 4
Inside 25 5
Inside 26 6
Inside 27 7
Inside 28 8
TimeoutException
change  3
TimeoutException
TimeoutException
TimeoutException
Inside 33 3
Inside 34 4
Inside 35 5
Inside 36 6
Inside 37 7
TimeoutException
Inside 39 9
change  4
Inside 40 0
Inside 41 1
TimeoutException
Inside 43 3
Inside 44 4
TimeoutException
Inside 46 6
TimeoutException
TimeoutException
TimeoutException
change  5
TimeoutException
Inside 51 1
Inside 52 2
Inside 53 3
Inside 54 4
TimeoutException
Inside 56 6
TimeoutException
TimeoutException
TimeoutException
change  6
Inside 60 0
Inside 61 1
Inside 62 2
Inside 63 3
Inside 64 4
Inside 65 5
Inside 66 6
Inside 67 7
TimeoutException
Inside 69 

TimeoutException: Message: 


In [53]:
# DataFrame

df = pd.DataFrame(news)
df

,titles,dates,par,font,url
0,"Hong Kong votes in overhauled ""patriots""-only ...","December 18, 2021",[],China,https://www.reuters.com/world/china/hong-kong-...
1,U.S. Air Force says two crew supporting Blinke...,"December 18, 2021",[],United States,https://www.reuters.com/world/us/us-air-force-...
2,U.N. rights forum agrees to investigate abuses...,"December 18, 2021",[],Africa,https://www.reuters.com/world/africa/ethiopia-...
3,Taiwan referendums fail in major setback for o...,"December 18, 2021",[],Commodities,https://www.reuters.com/markets/commodities/ta...
4,U.S. Senate confirms ex-Chicago Mayor Emanuel ...,"December 18, 2021",[],United States,https://www.reuters.com/world/us/us-senate-con...
5,Net profits of China's centrally-owned state f...,"December 18, 2021",[],China,https://www.reuters.com/world/china/net-profit...
6,Beijing city calls for less holiday travel to ...,"December 18, 2021",[],China,https://www.reuters.com/world/china/beijing-ci...
7,Explainer: How Hong Kong's new election law wi...,"December 18, 2021",[],China,https://www.reuters.com/world/china/how-hong-k...
8,EXCLUSIVE Hong Kong officials push turnout in ...,"December 18, 2021",[],Asia Pacific,https://www.reuters.com/world/asia-pacific/exc...
9,China's Alibaba pledges carbon neutrality by 2030,"December 17, 2021",[],Sustainable Business,https://www.reuters.com/business/sustainable-b...
